In [5]:
import datetime
import pytz
import yaml
import requests

from smolagents import CodeAgent, DuckDuckGoSearchTool, FinalAnswerTool, LiteLLMModel, load_tool, tool
from langchain_ollama.llms import OllamaLLM

In [3]:
@tool 
def my_custom_tool(arg1: str, arg2: int) -> str: # It's important to specify the return type.
    """A tool that does nothing yet...
    Args:
        arg1: the first argument
        arg2: the second argument
    """
    return "What magic will you build ?"


@tool
def get_current_time_in_timezone(timezone: str) -> str:
    """A tool that fetches the current local time in a specified timezone.
    Args:
        timezone: A string representing a valid timezone (e.g., 'America/New_York').
    """
    try:
        # Create timezone object
        tz = pytz.timezone(timezone)
        # Get current time in that timezone
        local_time = datetime.datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
        return f"The current local time in {timezone} is: {local_time}"
    except Exception as e:
        return f"Error fetching time for timezone '{timezone}': {str(e)}"


In [14]:
final_answer = FinalAnswerTool()

model = LiteLLMModel(model_id="ollama_chat/qwen2.5-coder:3b",
                     api_base="http://localhost:11434",)

# prompt template 
with open("prompts.yaml", 'r') as stream: 
    prompt_templates = yaml.safe_load(stream)

In [15]:
agent = CodeAgent(
    tools=[get_current_time_in_timezone, final_answer],
    model=model, 
    max_steps=6,
    verbosity_level=1,
    grammar=None,
    planning_interval=None,
    name=None,
    description=None,
    prompt_templates=prompt_templates
)

In [16]:
agent.run(task="what is the result of 1 + 1.00009 - 2.666 ?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ what is the result of 1 + 1.00009 - 2.666 ?                                                                     │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5-coder:3b ───────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  result = 1 + 1.00009 - 2.666                                                                                     
  final_answer(result)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: -0.6659099999999998

[Step 1: Duration 11.55 seconds| Input tokens: 2,097 | Output tokens: 54]

-0.6659099999999998

In [17]:
agent.run("what is current time in Kolkota ?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ what is current time in Kolkota ?                                                                               │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5-coder:3b ───────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  timezone = 'Asia/Kolkata'                                                                                        
  current_time_kolkata = get_current_time_in_timezone(timezone=timezone)                                           
  print("Current time in Kolkata:", current_time_kolkata)                                                          
  final_answer(current_time_kolkata)                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Current time in Kolkata: The current local time in Asia/Kolkata is: 2025-06-18 21:07:42

Out - Final answer: The current local time in Asia/Kolkata is: 2025-06-18 21:07:42

[Step 1: Duration 6.54 seconds| Input tokens: 2,082 | Output tokens: 73]

'The current local time in Asia/Kolkata is: 2025-06-18 21:07:42'